In [33]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf 
from tensorflow import keras

import math
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import LSTM 
from keras.layers import GRU
from keras.layers import Bidirectional,MaxPooling1D,Flatten,TimeDistributed
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error 
from keras.utils.np_utils import to_categorical 

from scipy.ndimage import zoom
%matplotlib inline

In [2]:
# Extract http://archive.ics.uci.edu/ml/machine-learning-databases/00447/ to same directory as this script

# List files
one = ['.\\data\\TS1.txt', '.\\data\\TS2.txt', '.\\data\\TS3.txt', '.\\data\\TS4.txt', '.\\data\\VS1.txt', '.\\data\\CE.txt', '.\\data\\CP.txt', '.\\data\\SE.txt']
ten = ['.\\data\\FS1.txt', '.\\data\\FS2.txt']
hundred = ['.\\data\\PS1.txt', '.\\data\\PS2.txt', '.\\data\\PS3.txt', '.\\data\\PS4.txt', '.\\data\\PS5.txt', '.\\data\\PS6.txt', '.\\data\\EPS1.txt']

# Parse condition profiles
df_profile = pd.read_table('.\\data\\profile.txt', header=None)
df_profile = df_profile.values.reshape(2205, 1, 5)
df_profile = zoom(df_profile, (1,6000,1))

# Parse 1 Hz measurements
df_one =  np.stack([pd.read_table(x, header=None) for x in one], axis=2)
df_one = zoom(df_one, (1, 100, 1))

# Parse 10 Hz measurements
df_ten =  np.stack([pd.read_table(x, header=None) for x in ten], axis=2)
df_ten = zoom(df_ten, (1, 10, 1))

# Parse 100 Hz measurements
df_hundred = np.stack([pd.read_table(x, header=None) for x in hundred], axis=2)

# Concatenate all data
df = np.concatenate([df_profile, df_one, df_ten, df_hundred], axis=2)

In [20]:
# Split data into training, validation, and test sets
val = 0.2
test = 0.1
train = 1 - val - test

X_train = df[:int(train*df.shape[0])+1:,::,[i not in [1] for i in range(df.shape[2])]]
X_val = df[int(train*df.shape[0])+1:int(train*df.shape[0])+int(val*df.shape[0])+1:,::,[i not in [1] for i in range(df.shape[2])]]
X_test = df[int(train*df.shape[0])+int(val*df.shape[0])+1::,::,[i not in [1] for i in range(df.shape[2])]]

oh_target = (np.arange(df[:,0,1].max()+1) == df[:,0,1][...,None]).astype(int)
oh_target = np.delete(oh_target,np.where(~oh_target.any(axis=0))[0], axis=1)

y_train = oh_target[:int(train*oh_target.shape[0])+1:,]
y_val = oh_target[int(train*oh_target.shape[0])+1:int(train*oh_target.shape[0])+int(val*oh_target.shape[0])+1:,]
y_test = oh_target[int(train*oh_target.shape[0])+int(val*oh_target.shape[0])+1::,]

In [21]:
#verify shape for each dataset
X_train.shape,X_val.shape,X_test.shape,y_train.shape,y_val.shape,y_test.shape

((1544, 6000, 21),
 (441, 6000, 21),
 (220, 6000, 21),
 (1544, 4),
 (441, 4),
 (220, 4))

# Use Keras

In [22]:
# create and fit the first model
# LSTM RNN
model_1 = Sequential()
model_1.add(LSTM(4))
model_1.add(Dense(y_train.shape[1]))
model_1.add(Activation('softmax'))
model_1.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])


In [23]:
#fit model 1
history_1 = model_1.fit(X_train, y_train, epochs=10, batch_size=len(X_train), verbose=2)

Epoch 1/10
 - 31s - loss: 1.2951 - acc: 0.5337
Epoch 2/10
 - 21s - loss: 1.2943 - acc: 0.5337
Epoch 3/10
 - 21s - loss: 1.2935 - acc: 0.5337
Epoch 4/10
 - 18s - loss: 1.2927 - acc: 0.5337
Epoch 5/10
 - 19s - loss: 1.2920 - acc: 0.5337
Epoch 6/10
 - 19s - loss: 1.2912 - acc: 0.5337
Epoch 7/10
 - 20s - loss: 1.2904 - acc: 0.5337
Epoch 8/10
 - 19s - loss: 1.2896 - acc: 0.5337
Epoch 9/10
 - 19s - loss: 1.2888 - acc: 0.5337
Epoch 10/10
 - 18s - loss: 1.2880 - acc: 0.5337


In [24]:
#evaluation for model_1 validation dataset
score_model1=model_1.evaluate(x=X_val, y=y_val) 
# calcute accuracy and loss 
print('test loss: ', score_model1[0])
print('test accuracy: ', score_model1[1])

441/441 [==============================] - 5s 12ms/step
test loss:  1.2946663827312237
test accuracy:  0.5170068025859305


In [27]:
# create and fit the 2nd model
# GRU
model_2 = Sequential()
model_2.add(GRU(4))
model_2.add(Dense(y_train.shape[1]))
model_2.add(Activation('softmax'))
model_2.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [28]:
#fit model 2
history_2 = model_2.fit(X_train, y_train, epochs=10, batch_size=len(X_train), verbose=2)

Epoch 1/10
 - 15s - loss: 1.3863 - acc: 0.1554
Epoch 2/10
 - 15s - loss: 1.3857 - acc: 0.5337
Epoch 3/10
 - 15s - loss: 1.3852 - acc: 0.5337
Epoch 4/10
 - 16s - loss: 1.3847 - acc: 0.5337
Epoch 5/10
 - 16s - loss: 1.3842 - acc: 0.5337
Epoch 6/10
 - 17s - loss: 1.3836 - acc: 0.5337
Epoch 7/10
 - 18s - loss: 1.3830 - acc: 0.5337
Epoch 8/10
 - 17s - loss: 1.3825 - acc: 0.5337
Epoch 9/10
 - 16s - loss: 1.3819 - acc: 0.5337
Epoch 10/10
 - 18s - loss: 1.3813 - acc: 0.5337


In [29]:
#evaluation for model_2 validation dataset
score_model2=model_2.evaluate(x=X_val, y=y_val) 
# calcute accuracy and loss 
print('test loss: ', score_model2[0])
print('test accuracy: ', score_model2[1])

441/441 [==============================] - 5s 10ms/step
test loss:  1.3811186292544515
test accuracy:  0.5170068025859305


In [34]:
# create and fit the 3rd model
# BiLSTM + CNN

model_3 = Sequential() 
model_3.add(Bidirectional(LSTM(4, return_sequences=True)))
model_3.add(TimeDistributed(Dense(4)))
model_3.add(Activation('softplus'))
model_3.add(MaxPooling1D(5))
model_3.add(Flatten())
model_3.add(Dense(y_train.shape[1]))
model_3.add(Activation('softmax'))
model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
#fit model 3
history_3 = model_3.fit(X_train, y_train, epochs=10, batch_size=len(X_train), verbose=2)

Epoch 1/10
 - 221s - loss: 2.0441 - acc: 0.1554
Epoch 2/10
 - 272s - loss: 1.6295 - acc: 0.5337
Epoch 3/10
 - 200s - loss: 1.7415 - acc: 0.5337
Epoch 4/10
 - 193s - loss: 1.9198 - acc: 0.1554
Epoch 5/10
 - 183s - loss: 1.3111 - acc: 0.5337
Epoch 6/10
 - 170s - loss: 1.3548 - acc: 0.5337
Epoch 7/10
 - 157s - loss: 1.4960 - acc: 0.5337
Epoch 8/10
 - 121s - loss: 1.5506 - acc: 0.5337
Epoch 9/10
 - 151s - loss: 1.5491 - acc: 0.5337
Epoch 10/10
 - 168s - loss: 1.4701 - acc: 0.5337


In [36]:
#evaluation for model_3 validation dataset
score_model3=model_3.evaluate(x=X_val, y=y_val) 
# calcute accuracy and loss 
print('test loss: ', score_model3[0])
print('test accuracy: ', score_model3[1])

441/441 [==============================] - 9s 20ms/step
test loss:  1.3168306929183926
test accuracy:  0.5170068025859305


# Use Tensorflow